In [32]:
import csv
import math
import random




In [33]:
with open('Colonia.csv', newline='',encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    cities = []
    for row in reader:
        city = row['Cidade']
        x, y = map(int, (row['X'], row['Y']))
        cities.append((city, x, y))


In [34]:
def distCalculator(ponto1, ponto2):
    return math.sqrt((ponto1[1]-ponto2[1])**2 + (ponto1[2]-ponto2[2])**2)


In [35]:
grafo = []
initialPhero=1
for i in cities:
    arestas = []
    for j in cities:
        dist = distCalculator(i,j)
        if dist == 0:
            inverseDist = 0
        else:
            inverseDist = 1/distCalculator(i,j)
        arestas.append({
            'dist': dist,
            'phero': initialPhero,
            'invDist': inverseDist,
            'heuristic': initialPhero * inverseDist,
            'prob': 0
        })
    grafo.append(arestas)

grafo


[[{'dist': 0.0, 'phero': 1, 'invDist': 0, 'heuristic': 0, 'prob': 0},
  {'dist': 4.0, 'phero': 1, 'invDist': 0.25, 'heuristic': 0.25, 'prob': 0},
  {'dist': 8.0, 'phero': 1, 'invDist': 0.125, 'heuristic': 0.125, 'prob': 0},
  {'dist': 4.123105625617661,
   'phero': 1,
   'invDist': 0.24253562503633297,
   'heuristic': 0.24253562503633297,
   'prob': 0},
  {'dist': 2.23606797749979,
   'phero': 1,
   'invDist': 0.4472135954999579,
   'heuristic': 0.4472135954999579,
   'prob': 0},
  {'dist': 2.23606797749979,
   'phero': 1,
   'invDist': 0.4472135954999579,
   'heuristic': 0.4472135954999579,
   'prob': 0},
  {'dist': 5.0990195135927845,
   'phero': 1,
   'invDist': 0.19611613513818404,
   'heuristic': 0.19611613513818404,
   'prob': 0},
  {'dist': 2.0, 'phero': 1, 'invDist': 0.5, 'heuristic': 0.5, 'prob': 0},
  {'dist': 2.8284271247461903,
   'phero': 1,
   'invDist': 0.35355339059327373,
   'heuristic': 0.35355339059327373,
   'prob': 0},
  {'dist': 4.47213595499958,
   'phero': 1,
  

In [36]:
def atualizaHeur(grafo):
    alpha = 4.75
    beta = 5
    for i in grafo:
        for j in i:
            j['heuristic'] = (j['phero']**alpha) * (j['invDist']**beta)

def atualizaProb(grafo):
    for i in grafo:
        somaHeuristica = 0
        for j in i:
            somaHeuristica += j['heuristic']
        for j in i:
            j['prob']= j['heuristic']/somaHeuristica


In [37]:
atualizaProb(grafo)



In [38]:
def roleta(cidadeAtual,javisitados):
    roleta = []
    soma = 0
    for j in range((len(cities))):
        if j not in javisitados:
            roleta.append((j,grafo[cidadeAtual][j]['prob']+soma))
            soma+=grafo[cidadeAtual][j]['prob']
    gerado = random.uniform(0,roleta[-1][1])
    for j in roleta:
        if gerado <= j[1]:
            return j[0]


In [39]:
def caminhar():
    pathsAndDist = []
    for i in range(len(cities)):
        javisitados = {i:1}
        cidadeAtual = i
        distanciaPercorrida = 0
        path = [i]
        while (len(javisitados)<len(cities)):
            selecionado = roleta(cidadeAtual,javisitados)
            javisitados[selecionado] = 1
            path.append(selecionado)
            distanciaPercorrida += grafo[cidadeAtual][selecionado]['dist']
            cidadeAtual = selecionado

        distanciaPercorrida+= grafo[cidadeAtual][i]['dist']
        path.append(i)
        pathsAndDist.append((path,distanciaPercorrida))
    return pathsAndDist

In [40]:
def atualizaFero(pathsAndDist):
    evaporacao = 0.05
    pheroAmount = 0.0000001
    for i in pathsAndDist:
        path = i[0]
        dist = i[1]
        pheroPorRota = pheroAmount/dist
        for j in range(1,len(path)):
            grafo[path[j]][path[j-1]]['phero'] = (1-evaporacao)*grafo[path[j]][path[j-1]]['phero'] + pheroPorRota
            grafo[path[j-1]][path[j]]['phero'] = (1-evaporacao)*grafo[path[j-1]][path[j]]['phero'] + pheroPorRota
            



In [41]:
data = []
for i in range(300):
    pathsAndDist = caminhar()
    print(pathsAndDist)
    data.append(sum([i[1] for i in pathsAndDist])/len(pathsAndDist))
    atualizaFero(pathsAndDist)
    atualizaHeur(grafo)
    atualizaProb(grafo)

data
    
    

[([0, 5, 3, 13, 4, 22, 2, 21, 16, 15, 28, 19, 20, 23, 24, 10, 31, 17, 7, 29, 9, 30, 27, 12, 11, 1, 14, 6, 25, 18, 8, 26, 0], 160.03215411499554), ([1, 15, 7, 12, 13, 19, 21, 29, 10, 8, 28, 31, 25, 26, 17, 14, 20, 27, 16, 4, 2, 0, 23, 3, 5, 6, 9, 24, 30, 11, 22, 18, 1], 152.84840316990235), ([2, 19, 20, 7, 16, 11, 22, 1, 3, 14, 8, 23, 27, 5, 18, 6, 17, 15, 13, 0, 9, 28, 10, 25, 12, 4, 31, 26, 29, 21, 30, 24, 2], 170.27611357334354), ([3, 4, 9, 29, 5, 2, 6, 18, 0, 13, 12, 24, 28, 21, 15, 1, 8, 14, 7, 16, 27, 20, 26, 23, 22, 17, 19, 30, 31, 25, 10, 11, 3], 131.68277005261697), ([4, 5, 12, 11, 22, 16, 0, 29, 6, 10, 18, 9, 17, 20, 26, 28, 23, 24, 7, 21, 31, 15, 14, 8, 1, 2, 25, 3, 30, 27, 13, 19, 4], 148.69637210546367), ([5, 28, 14, 19, 1, 21, 30, 2, 12, 20, 31, 29, 7, 0, 24, 27, 17, 15, 10, 9, 22, 25, 18, 4, 8, 16, 13, 6, 23, 26, 11, 3, 5], 155.18596911847862), ([6, 5, 13, 4, 11, 19, 26, 28, 24, 16, 23, 1, 15, 22, 17, 8, 14, 10, 12, 21, 0, 30, 27, 2, 31, 29, 25, 20, 3, 18, 9, 7, 6], 143.2

[152.14156619009955,
 176.24624157510078,
 104.02347561327264,
 80.93860771883412,
 79.8346160108979,
 76.96595156511084,
 80.14588339384284,
 78.04412381729605,
 79.1170282265276,
 78.52705671461331,
 81.1530795398079,
 79.69959377701306,
 80.94549547728211,
 81.06637739624705,
 77.39626104557584,
 80.7432440229898,
 81.87476696768194,
 81.63917188101918,
 83.08647363579303,
 81.77015585201794,
 82.94363504516345,
 81.5066979463773,
 81.15571741821347,
 81.21391214965432,
 79.26353863336415,
 81.84251323660301,
 79.69400912969219,
 79.46346019540029,
 82.14040438481246,
 80.96940185628347,
 82.80434405733307,
 80.48043885515045,
 80.62715751954286,
 84.72383552707146,
 80.73001673866463,
 78.91336358484594,
 81.59788506530914,
 79.10685793395619,
 83.34267136715863,
 79.37748353764458,
 79.10092044295247,
 82.20606378488927,
 81.79420759886631,
 80.74625703868985,
 78.15249039263271,
 84.22136732919394,
 80.75175590918892,
 81.28403298075546,
 82.09597532235912,
 82.26191691123054,
 8